In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("../data/processed/covtype_explored.csv")

In [3]:
df.head(3)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,Wilderness_Area_Label,Wilderness_Area_Code,Soil_Type_Code,Soil_Type_Label
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,5,Wilderness_Area1,1,29,Soil_Type29
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,5,Wilderness_Area1,1,29,Soil_Type29
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,2,Wilderness_Area1,1,12,Soil_Type12


In [4]:
num_cols = [
    "Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Roadways",
    "Hillshade_9am","Hillshade_Noon","Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
]

wilderness_cols = [f"Wilderness_Area{i}" for i in range(1,5)]
soil_cols = [f"Soil_Type{i}" for i in range(1,41)]
target_col = ["Cover_Type"]
wilderness_type_col = ['Wilderness_Area_Code']
soil_type_col = ['Soil_Type_Code']
cat_cols = ["Wilderness_Area_Label","Soil_Type_Label"]


In [5]:
df.drop(columns=cat_cols + wilderness_type_col + soil_type_col, inplace=True)

In [6]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [8]:
import numpy as np

df['hydro_distance'] = np.sqrt(
    df['Horizontal_Distance_To_Hydrology']**2 + 
    df['Vertical_Distance_To_Hydrology']**2
)
df.drop(columns=['Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology'], inplace=True)

In [9]:
from sklearn.model_selection import train_test_split


X = df.drop(target_col, axis=1)
y = df[target_col]


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)


X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)


# train_df = X_train.copy()
# train_df["Cover_Type"] = y_train

# val_df = X_val.copy()
# val_df["Cover_Type"] = y_val

# test_df = X_test.copy()
# test_df["Cover_Type"] = y_test


# train_df.to_csv("../data/train_split.csv", index=False)
# val_df.to_csv("validation_split.csv", index=False)
# test_df.to_csv("test_split.csv", index=False)

# print("Saved train/validation/test CSV files:")
# print(" - train_split.csv")
# print(" - validation_split.csv")
# print(" - test_split.csv")


In [10]:
from sklearn.preprocessing import StandardScaler
import joblib   

scaler = StandardScaler()

# Fit only on training set
scaler.fit(X_train[num_cols])

X_train_scaled = X_train.copy()
X_val_scaled   = X_val.copy()
X_test_scaled  = X_test.copy()

# Replace only numeric columns with scaled ones
X_train_scaled[num_cols] = scaler.transform(X_train[num_cols])
X_val_scaled[num_cols]   = scaler.transform(X_val[num_cols])
X_test_scaled[num_cols]  = scaler.transform(X_test[num_cols])


KeyError: "['Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology'] not in index"

In [ ]:
joblib.dump(scaler, "../models/standard_scaler.joblib")


train_df = X_train_scaled.copy()
train_df["Cover_Type"] = y_train

val_df = X_val_scaled.copy()
val_df["Cover_Type"] = y_val

test_df = X_test_scaled.copy()
test_df["Cover_Type"] = y_test


train_df.to_csv("../data/interim/train_f.csv", index=False)
val_df.to_csv("../data/interim/validation_f.csv", index=False)
test_df.to_csv("../data/interim/test_f.csv", index=False)




In [ ]:
# scaler = joblib.load("standard_scaler.joblib")

# new_data_scaled = scaler.transform(new_data[num_cols])
